In [4]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd.take(1)
splitedRdd = sc.parallelize(splitedRdd.take(20))

splitedRdd.take(1)
#(uid,[[train],[test]]

[(61952,
  [[train(itemId=60075, context=context(ts=1421371713, city=12940, lat=43.503536, lon=-88.558907, moving=1, location=3, time_of_day=4)),
    train(itemId=91914, context=context(ts=1421371762, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4)),
    train(itemId=91512, context=context(ts=1421371807, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4)),
    train(itemId=91914, context=context(ts=1421371816, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4)),
    train(itemId=43924, context=context(ts=1421371837, city=12940, lat=43.50354, lon=-88.55899, moving=2, location=3, time_of_day=4)),
    train(itemId=91914, context=context(ts=1421371838, city=12940, lat=43.50354, lon=-88.55899, moving=2, location=3, time_of_day=4)),
    train(itemId=44053, context=context(ts=1421371861, city=12940, lat=43.50354, lon=-88.55899, moving=2, location=3, time_of_day=4)),
    train(itemId=91914, context=contex

In [11]:
ContextRow2 = namedtuple("context", ["ts","city", "lat", "lon", "moving", "location", "time_of_day",
                                     "last_app", "app_chain" ])
def addLastApp(row):
    train = row[1][0]
    test = row[1][1]
    if (len(train) == 0 or len(test) == 0):
        #cannot evaluate
        return (row[0], [[],[]])
    
    temp_list = list(train[0])
    temp_list[1] = list(temp_list[1])
    temp_list[1].append(-1) #last app unknown
    temp_list[1].append(-1) #app chain unknown
    new_row = [[TrainRow(temp_list[0],ContextRow2(*temp_list[1]))]]
    
    ts = test[0].context.ts - train[-1].context.ts
    app_chain_temp = -1 if ts > 60 else train[-1].itemId
    temp_list = list(test[0])
    temp_list[1] = list(temp_list[1])
    temp_list[1].append(train[-1].itemId) #last app 
    temp_list[1].append(app_chain_temp) #app chain
    new_row.append([TrainRow(temp_list[0],ContextRow2(*temp_list[1]))])
    for i in xrange(1,len(train)):
        ts = train[i].context.ts - train[i-1].context.ts
        app_chain_temp = -1 if ts > 60 else train[i-1].itemId
        temp_list = list(train[i])
        temp_list[1] = list(temp_list[1])
        temp_list[1].append(train[i-1].itemId) #last app 
        temp_list[1].append(app_chain_temp) #app chain 
        new_row[0].append(
            TrainRow(temp_list[0], ContextRow2(*temp_list[1]))
        )
    for i in xrange(1,len(test)):
        ts = test[i].context.ts - test[i-1].context.ts
        app_chain_temp = -1 if ts > 60 else test[i-1].itemId
        temp_list = list(test[i])
        temp_list[1] = list(temp_list[1])
        temp_list[1].append(test[i-1].itemId) #last app 
        temp_list[1].append(app_chain_temp) #app chain 
        new_row[1].append(
            TrainRow(temp_list[0], ContextRow2(*temp_list[1]))
        )
    return (row[0], new_row)
        
splitedRddv2 = splitedRdd.map(addLastApp)
splitedRddv2.take(1)

[(61952,
  [[train(itemId=60075, context=context(ts=1421371713, city=12940, lat=43.503536, lon=-88.558907, moving=1, location=3, time_of_day=4, last_app=-1, app_chain=-1)),
    train(itemId=91914, context=context(ts=1421371762, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4, last_app=60075, app_chain=60075)),
    train(itemId=91512, context=context(ts=1421371807, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4, last_app=91914, app_chain=91914)),
    train(itemId=91914, context=context(ts=1421371816, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4, last_app=91512, app_chain=91512)),
    train(itemId=43924, context=context(ts=1421371837, city=12940, lat=43.50354, lon=-88.55899, moving=2, location=3, time_of_day=4, last_app=91914, app_chain=91914)),
    train(itemId=91914, context=context(ts=1421371838, city=12940, lat=43.50354, lon=-88.55899, moving=2, location=3, time_of_day=4, last_app=43924, 

In [17]:
splitedRdd_flat = splitedRddv2.flatMap(lambda x: x[1]).flatMap(lambda x: x)
splitedRdd_flat.take(2)

[train(itemId=60075, context=context(ts=1421371713, city=12940, lat=43.503536, lon=-88.558907, moving=1, location=3, time_of_day=4, last_app=-1, app_chain=-1)),
 train(itemId=91914, context=context(ts=1421371762, city=12940, lat=43.503551, lon=-88.558922, moving=2, location=3, time_of_day=4, last_app=60075, app_chain=60075))]

In [56]:
numAll = splitedRdd_flat.count()
all_context = splitedRdd_flat.map(lambda x: x[1][8]).map(lambda x:(x,1.0)).groupByKey().map(lambda x:(x[0],sum(x[1])/numAll))
all_context.take(10)

[(45644, 8.999280057595392e-05),
 (47296, 8.999280057595392e-05),
 (74760, 0.00044996400287976963),
 (75180, 8.999280057595392e-05),
 (55752, 0.0002699784017278618),
 (79500, 8.999280057595392e-05),
 (74000, 8.999280057595392e-05),
 (12184, 0.0008099352051835853),
 (18068, 0.00017998560115190784),
 (98232, 8.999280057595392e-05)]

In [58]:
all_appid = splitedRdd_flat.map(lambda x: x[0]).map(lambda x:(x,1.0)).groupByKey().map(lambda x:(x[0],sum(x[1])/numAll))
all_appid.take(10)

[(45644, 8.999280057595392e-05),
 (11712, 0.0003599712023038157),
 (29844, 8.999280057595392e-05),
 (66180, 8.999280057595392e-05),
 (80880, 0.00017998560115190784),
 (74760, 0.00044996400287976963),
 (75180, 0.00017998560115190784),
 (55752, 0.0007199424046076314),
 (27788, 0.005039596832253419),
 (56820, 8.999280057595392e-05)]

In [60]:
item_context = splitedRdd_flat.map(lambda x: (x[0],x[1][8]))
item_context.take(10)

[(60075, -1),
 (91914, 60075),
 (91512, 91914),
 (91914, 91512),
 (43924, 91914),
 (91914, 43924),
 (44053, 91914),
 (91914, 44053),
 (91914, -1),
 (54178, 91914)]

In [49]:
numRecList = 5
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores

def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation2.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()

In [ ]:
def bayesian_context_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_moving)
print_result(finalScore, 'bayesian_context_moving')

In [ ]:
def bayesian_context_location(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5]) for t in trainList] #itemId,locationId
    testList = [(t[0],t[1][5]) for t in testList]   #itemId,locationID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_location)
print_result(finalScore, 'bayesian_context_location')

In [ ]:
def bayesian_context_time(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][6]) for t in trainList] #itemId,timeId
    testList = [(t[0],t[1][6]) for t in testList]   #itemId,timeID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_time)
print_result(finalScore, 'bayesian_context_time')

In [ ]:
def bayesian_context_last_app(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][7]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][7]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_last_app)
print_result(finalScore, 'bayesian_context_last_app')

In [35]:
def bayesian_context_last_app_chain(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][8]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_last_app_chain)
print_result(finalScore, 'bayesian_context_last_app_chain')

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 98.0 failed 1 times, most recent failure: Lost task 3.0 in stage 98.0 (TID 718, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2330, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2330, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/pyth

In [39]:
def bayesian_context_last_app_chain_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][8]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if (x[1]==testList[0][1] or x[2]==testList[0][2])]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and (x[1]==c[1] or x[2]==c[2]) ]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_last_app_chain_moving)
print_result(finalScore, 'bayesian_context_last_app_chain_moving')

In [42]:
def bayesian_context_last_app_chain_movin2(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][8]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)


    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1] ]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            if testList[0][2] == c[2] and c[2]!=-1:
                p = p * 1.2
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_last_app_chain_movin2)
print_result(finalScore, 'bayesian_context_last_app_chain_movin2')

In [ ]:
def bayesian_context_combine(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in trainList] #itemId,movingId,locationID,timeID
    testList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
    for i in range(numTest):
        #moving
        context = [x for x in trainList if x[1]!=1 and x[1]==testList[0][1]] # x[1]!=1 moving is not avaiable
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]!=1 and x[1]==c[1] ] # x[1]!=1 moving is not avaiable
            if numAppi == 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        #location  
        context = [x for x in trainList if x[2]!=3 and x[2]==testList[0][2]] # x[2]!=3 location else where
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[2]!=3 and x[2]==c[2] ] # x[2]!=3 location else where
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p) 
        
        #last_app_chain 
        context = [x for x in trainList if x[5]!=-1 and x[5]==testList[0][5]] # x[5]!=-1 last app not chain when is -1
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[5]!=-1 and x[5]==c[5] ] # x[5]!=-1 last app not chain when is -1
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p) 
        
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRddv2.map(bayesian_context_combine)
print_result(finalScore, 'bayesian_context_combine')


In [64]:
def bayesian_context_combine2(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in trainList] #itemId,movingId,locationID,timeID
    testList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
    for i in range(numTest):
        #moving
        #context1 = [(x[0],x[1]) for x in trainList if x[1]==testList[0][1]] # x[1]!=1 moving is not avaiable
        numContext1 = float(len([(x[0],x[1]) for x in trainList if x[1]==testList[0][1]]))
        if numTrain != 0:
            p_context1 = numContext1/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context1 = 0
        
        #context5 = [(x[0],x[5]) for x in trainList if x[5]==testList[0][5]] # x[5]!=-1 last app not chain when is -1
        numContext5 = float(len([(x[0],x[5]) for x in trainList if x[5]==testList[0][5]]))
        if numTrain != 0:
            p_context5 = numContext5/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context5 = 0
        #context = [(x[0],x[1],x[5]) for x in trainList if x[1]==testList[0][1] or x[5]==testList[0][5]] # x[1]!=1 moving is not avaiable    
        context_no_duplicate = remove_duplicates([(x[0],x[1],x[5]) for x in trainList if x[1]==testList[0][1] or x[5]==testList[0][5]])
        
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
                
            #contextAppi1 = [x for x in trainList if x[0]==c[0] and x[1]==c[1] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi1 = len(contextAppi1)/numAppi 
                p_contextAppi1 = len([x for x in trainList if x[0]==c[0] and x[1]==c[1] ])/numAppi 
            else:
                p_contextAppi1 = 0
            if p_context1 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p1 = p_contextAppi1 * p_appi / p_context1
            else:
                p1 = 0
                
            #contextAppi5 = [x for x in trainList if x[0]==c[0] and x[5]==c[2] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi5 = len(contextAppi5)/numAppi 
                p_contextAppi5 = len([x for x in trainList if x[0]==c[0] and x[5]==c[2] ])/numAppi 
            else:
                p_contextAppi5 = 0
            if p_context5 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p5 = p_contextAppi5 * p_appi / p_context5
            else:
                p5 = 0    

            p_app = topFiveSortedList(p_app,c[0],p1*p5)
        
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
    #return contextAppi1
finalScore = splitedRddv2.map(bayesian_context_combine2)
print_result(finalScore, 'bayesian_context_combine2')


In [65]:
def bayesian_context_combine3(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in trainList] #itemId,movingId,locationID,timeID
    testList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
    for i in range(numTest):
        #moving
        #context1 = [(x[0],x[1]) for x in trainList if x[1]==testList[0][1]] # x[1]!=1 moving is not avaiable
        numContext1 = float(len([(x[0],x[1]) for x in trainList if x[1]==testList[0][1]]))
        if numTrain != 0:
            p_context1 = numContext1/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context1 = 0
        
        #context5 = [(x[0],x[5]) for x in trainList if x[5]==testList[0][5]] # x[5]!=-1 last app not chain when is -1
        numContext5 = float(len([(x[0],x[5]) for x in trainList if x[5]==testList[0][5]]))
        if numTrain != 0:
            p_context5 = numContext5/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context5 = 0
        #context = [(x[0],x[1],x[5]) for x in trainList if x[1]==testList[0][1] or x[5]==testList[0][5]] # x[1]!=1 moving is not avaiable    
        context_no_duplicate = remove_duplicates([(x[0],x[1],x[5]) for x in trainList if x[1]==testList[0][1] or x[5]==testList[0][5]])
        
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
                
            #contextAppi1 = [x for x in trainList if x[0]==c[0] and x[1]==c[1] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi1 = len(contextAppi1)/numAppi 
                p_contextAppi1 = len([x for x in trainList if x[0]==c[0] and x[1]==c[1] ])/numAppi 
            else:
                p_contextAppi1 = 0
            if p_context1 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p1 = p_contextAppi1 * p_appi / p_context1
            else:
                p1 = 0
                
            #contextAppi5 = [x for x in trainList if x[0]==c[0] and x[5]==c[2] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi5 = len(contextAppi5)/numAppi 
                p_contextAppi5 = len([x for x in trainList if x[0]==c[0] and x[5]==c[2] ])/numAppi 
            else:
                p_contextAppi5 = 0
            if p_context5 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p5 = p_contextAppi5 * p_appi / p_context5
            else:
                p5 = 0    

            p_app = topFiveSortedList(p_app,c[0],p1+p5)
        
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
    #return contextAppi1
finalScore = splitedRddv2.map(bayesian_context_combine3)
print_result(finalScore, 'bayesian_context_combine3')


In [67]:
def bayesian_context_combine4(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in trainList] #itemId,movingId,locationID,timeID
    testList = [(t[0],t[1][4],t[1][5],t[1][6],t[1][7],t[1][8]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
    for i in range(numTest):
        #moving
        #context1 = [(x[0],x[1]) for x in trainList if x[1]==testList[0][1]] # x[1]!=1 moving is not avaiable
        numContext1 = float(len([x for x in trainList if x[1]==testList[0][1]]))
        if numTrain != 0:
            p_context1 = numContext1/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context1 = 0
        
        #location
        #context2 = [(x[0],x[2]) for x in trainList if x[2]==testList[0][2]] 
        numContext2 = float(len([x for x in trainList if x[2]==testList[0][2]]))
        if numTrain != 0:
            p_context2 = numContext2/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context2 = 0
        
        #last_app_chain
        #context5 = [(x[0],x[5]) for x in trainList if x[5]==testList[0][5]] # x[5]!=-1 last app not chain when is -1
        numContext5 = float(len([(x[0],x[5]) for x in trainList if x[5]==testList[0][5]]))
        if numTrain != 0:
            p_context5 = numContext5/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context5 = 0
        #context = [(x[0],x[1],x[5]) for x in trainList if x[1]==testList[0][1] or x[5]==testList[0][5]] # x[1]!=1 moving is not avaiable    
        context_no_duplicate = remove_duplicates([(x[0],x[1],x[2],x[5]) for x in trainList if x[1]==testList[0][1] or x[2]==testList[0][2] or x[5]==testList[0][5]])
        
        for c in context_no_duplicate:
            #appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len([x for x in trainList if x[0]==c[0]]))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            
            #contextAppi1 = [x for x in trainList if x[0]==c[0] and x[1]==c[1] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi1 = len(contextAppi1)/numAppi 
                p_contextAppi1 = len([x for x in trainList if x[0]==c[0] and x[1]==c[1] ])/numAppi 
            else:
                p_contextAppi1 = 0
            if p_context1 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p1 = p_contextAppi1 * p_appi / p_context1
            else:
                p1 = 0
                
            #contextAppi2 = [x for x in trainList if x[0]==c[0] and x[2]==c[2] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi2 = len(contextAppi2)/numAppi 
                p_contextAppi2 = len([x for x in trainList if x[0]==c[0] and x[2]==c[2] ])/numAppi 
            else:
                p_contextAppi2 = 0
            if p_context2 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p2 = p_contextAppi2 * p_appi / p_context2
            else:
                p2 = 0
                
            #contextAppi5 = [x for x in trainList if x[0]==c[0] and x[5]==c[2] ] # x[1]!=1 moving is not avaiable
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                #p_contextAppi5 = len(contextAppi5)/numAppi 
                p_contextAppi5 = len([x for x in trainList if x[0]==c[0] and x[5]==c[3] ])/numAppi 
            else:
                p_contextAppi5 = 0
            if p_context5 != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p5 = p_contextAppi5 * p_appi / p_context5
            else:
                p5 = 0    

            p_app = topFiveSortedList(p_app,c[0],p1*p2*p5)
        
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)     
        trainList.append(testList[0])
        testList.pop(0)       
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
    return scores_rr, scores_ap, scores_recall
    #return contextAppi1
finalScore = splitedRddv2.map(bayesian_context_combine4)
print_result(finalScore, 'bayesian_context_combine4')
